In [1]:
!pip install -q --upgrade pip
!pip install -q tensorflow-gpu==2.2.0
!pip install -q transformers

     |████████████████████████████████| 1.5MB 8.5MB/s 
     |████████████████████████████████| 516.2 MB 20 kB/s 
     |████████████████████████████████| 674 kB 7.9 MB/s 
     |████████████████████████████████| 1.1 MB 24.3 MB/s 
     |████████████████████████████████| 3.8 MB 56.5 MB/s 
     |████████████████████████████████| 883 kB 79.3 MB/s 


In [2]:
!git clone https://github.com/rgrupesh/Deep_Learning.git

Cloning into 'Deep_Learning'...
remote: Enumerating objects: 53, done.
remote: Counting objects: 100% (53/53), done.
remote: Compressing objects: 100% (46/46), done.
remote: Total 53 (delta 22), reused 17 (delta 7), pack-reused 0
Unpacking objects: 100% (53/53), done.


In [3]:
%cd 

/root


In [0]:
import tensorflow as tf
import pandas as pd
import numpy as np
from tensorflow.python.lib.io.tf_record import TFRecordWriter

In [5]:
train=pd.read_csv("/content/Deep_Learning/data/train.csv")
train.reset_index(inplace=True)
train["sentiment"].replace({"pos":1,"neg":0}, inplace=True)
train[5000:5005]

,index,text,sentiment
5000,5000,Lock Up Your Daughters is one of the best high...,1
5001,5001,"Woody Allen, when at his best, has one of the ...",0
5002,5002,You got to go and dig those holes. Holes only ...,1
5003,5003,This film is like a 1950-version of Ettore Sco...,1
5004,5004,A somewhat fictionalized biographical portrait...,1


In [6]:
test=pd.read_csv("/content/Deep_Learning/data/test.csv")
test.reset_index(inplace=True)
test["sentiment"].replace({"pos":1,"neg":0}, inplace=True)
test.tail()

,index,text,sentiment
24995,24995,"For one thing, he produced this movie. It has ...",0
24996,24996,The title comes from an alteration an adolesce...,1
24997,24997,Christopher Nolan's first film is a 'no budget...,1
24998,24998,The story is shortly about the faith-lacking b...,0
24999,24999,"I found parts of this movie rather slow, espec...",1


In [0]:
SAMPLE_F=0.2
TRAIN_F=0.8

train_sample=train.sample(frac=SAMPLE_F, random_state=0)

train_select=train_sample.sample(frac=TRAIN_F, random_state=0)
train_csv= train_select.values

validate_select=train_sample.drop(index=train_select.index)
validate_csv= validate_select.values

test_sample=test.sample(frac=SAMPLE_F, random_state=0)
test_csv=test_sample.values


In [0]:
def create_example(features,label):
      tf_example = tf.train.Example(features = tf.train.Features(feature = {
        'idx': tf.train.Feature(int64_list=tf.train.Int64List(value=[features[0]])),
        'sentence': tf.train.Feature(bytes_list=tf.train.BytesList(value=[features[1].encode('utf-8')])),
        'label': tf.train.Feature(int64_list=tf.train.Int64List(value=[label]))
    }))
      return tf_example.SerializeToString()

In [0]:
import time

def csv_to_tfRecord(csv, file_name):

  start_time = time.time()
  writer=TFRecordWriter(file_name)
  for index, row in enumerate(csv):
    features, label= row[:-1], row[-1]
    example=create_example(features, label)
    writer.write(example)
  writer.close()
  print(f"{file_name}:=====>>>{time.time() - start_time} secs")   

In [11]:
csv_to_tfRecord(validate_csv,"validate.tfrecord")
csv_to_tfRecord(train_csv,"train.tfrecord")
csv_to_tfRecord(test_csv,"test.tfrecord")

validate.tfrecord:=====>>>0.04120922088623047 secs
train.tfrecord:=====>>>0.13268733024597168 secs
test.tfrecord:=====>>>0.15041470527648926 secs


In [0]:
import json

def generate_json(file_name,train_df=[],val_df=[], test_df=[]):
  info={"train length": len(train_df),"val_length": len(val_df), "test_length": len(test_df)}

  with open(file_name,"w") as outfile:
    json.dump(info,outfile)

generate_json("info.json", train_csv, validate_csv, test_csv)    

In [0]:
import tensorflow as tf
from transformers import *
from transformers import BertTokenizer, TFBertForSequenceClassification, glue_convert_examples_to_features
from transformers.configuration_bert import BertConfig

In [0]:
train_ds=tf.data.TFRecordDataset("train.tfrecord")
val_ds= tf.data.TFRecordDataset("validate.tfrecord")
test_ds= tf.data.TFRecordDataset("test.tfrecord")

In [0]:
feature_spec = {
    'idx': tf.io.FixedLenFeature([], tf.int64),
    'sentence': tf.io.FixedLenFeature([], tf.string),
    'label': tf.io.FixedLenFeature([], tf.int64)
}
def parse_example(example_proto):
    return tf.io.parse_single_example(example_proto, feature_spec)

train_parse_ds=train_ds.map(parse_example)
val_parse_ds=val_ds.map(parse_example)
test_parse_ds=test_ds.map(parse_example)


In [0]:
def clean_string(features):
    revised = tf.strings.regex_replace(features['sentence'], "\.\.\.", "", replace_global=True)
    revised = tf.strings.regex_replace(revised, "\\'", "'", replace_global=True)
    revised = tf.strings.regex_replace(revised, "\\n", "", replace_global=True)
    features["sentence"]=revised
    return features

train_clean_ds = train_parse_ds.map(lambda features: clean_string(features))
val_clean_ds = val_parse_ds.map(lambda features: clean_string(features))
test_clean_ds =  test_parse_ds.map(lambda features: clean_string(features))    


In [0]:
with open("info.json") as json_file:
  info=json.load(json_file)

train_examples=info["train length"]
val_examples=info["val_length"]  
test_examples=info["test_length"]

In [18]:
train_examples, val_examples,test_examples

(4000, 1000, 5000)

In [19]:
NUM_LABELS=2

config=BertConfig.from_pretrained("bert-base-cased", num_labels=NUM_LABELS)
tokenizer=BertTokenizer.from_pretrained("bert-base-cased")
model= TFBertForSequenceClassification.from_pretrained("bert-base-cased",config=config)

In [0]:
train_dataset = glue_convert_examples_to_features(examples=train_clean_ds, tokenizer=tokenizer
                                                  , max_length=512, task='sst-2',
                                                  label_list=['0','1']
                                                  )

In [0]:
val_dataset = glue_convert_examples_to_features(examples=val_clean_ds, tokenizer=tokenizer
                                                  , max_length=512, task='sst-2',
                                                  label_list=['0','1']
                                                  )

In [0]:
BATCH_SIZE= 6
EVAL_BATCH_SIZE= 12

train_dataset= train_dataset.shuffle(train_examples).batch(BATCH_SIZE).repeat(-1)

val_dataset=val_dataset.batch(EVAL_BATCH_SIZE)


In [0]:
optimizer=tf.keras.optimizers.Adam(lr=3e-5,epsilon=1e-8)
loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
metric=tf.keras.metrics.SparseCategoricalAccuracy("accuracy")

model.compile(optimizer=optimizer,loss=loss,metrics=[metric])

In [0]:
train_steps= train_examples // BATCH_SIZE
val_steps= val_examples // EVAL_BATCH_SIZE

In [25]:
model.summary()

Model: "tf_bert_for_sequence_classification"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bert (TFBertMainLayer)       multiple                  108310272 
_________________________________________________________________
dropout_37 (Dropout)         multiple                  0         
_________________________________________________________________
classifier (Dense)           multiple                  1538      
Total params: 108,311,810
Trainable params: 108,311,810
Non-trainable params: 0
_________________________________________________________________


In [26]:
history= model.fit(train_dataset, epochs=1, validation_data=val_dataset, verbose=2, steps_per_epoch=train_steps, validation_steps= val_steps)

666/666 - 557s - loss: 0.4769 - accuracy: 0.7663 - val_loss: 0.2764 - val_accuracy: 0.8916


In [0]:
test_dataset = glue_convert_examples_to_features(examples=test_clean_ds, tokenizer=tokenizer
                                                  , max_length=512, task='sst-2'
                                                  , label_list =['0', '1'])

In [0]:
test_dataset = test_dataset.batch(EVAL_BATCH_SIZE)

In [29]:
history1=model.evaluate(test_dataset)

417/417 [==============================] - 206s 495ms/step - loss: 0.2714 - accuracy: 0.8898


In [41]:
tf.saved_model.save(model, 'BERT-SST-2')

INFO:tensorflow:Assets written to: BERT-SST-2/assets
